Implementation Plan:

You can run git commands like this to get the latest version of code to work with:

git checkout preprod && git pull && git checkout -B uc-code-refactor-20230225 && git checkout uc-code-refactor-20230224 ./uc1Codefix.ipynb

In [0]:
import os, re, fnmatch

Function to find a given string and replace all occurances in all the notebooks (.py) in the current direcotry and its sub directories

In [0]:
def find_replace(directory, filename_filter, find, replace, filePattern="*.py") -> None:
    """traverses through the given directory and using given conditions and pattern, does a find + replace"""
    for path, dirs, files in os.walk(os.path.abspath(directory)):
        print(path)
        for filename in fnmatch.filter(files, filePattern):
            if filename_filter.lower() in filename.lower():
                filepath = os.path.join(path, filename)
                with open(filepath) as f:
                    s = f.read()
                if find.lower() in s.lower():
                    s = s.replace(find, replace)
                    print (f"{filename} updated - [{find}] --> [{replace}]")
                with open(filepath, "w") as f:
                    f.write(s)


**MDP View Notebooks' Refactoring**

Some view notebooks have references that deviate from the standard naming convention, hence handling manually in the following command

In [0]:
directory   = "../Transform/Views/"

findstr     = 'cleansed.vw_swirl_ref_lookup'
replacestr  = '{get_table_namespace(\'cleansed\', \'swirl_ref_lookup\')}'
find_replace(directory, 'swirl-view-creation.py', findstr, replacestr)

findstr     = 'cleansed.vw_swirl_ref_nfcomponents'
replacestr  = '{get_table_namespace(\'cleansed\', \'swirl_ref_nfcomponents\')}'
find_replace(directory, 'swirl-view-creation.py', findstr, replacestr)

findstr     = 'cleansed.crm_zpstxhwithcguid'
replacestr  = '{get_table_namespace(\'cleansed\', \'crm_zpstxhwithcguid\')}'
find_replace(directory, 'WorkNote.py', findstr, replacestr)

findstr     = 'GetTable(f"{SOURCE}.crm_zcs_long_text_f")'
replacestr  = 'GetTable(f"{get_table_namespace(f'{SOURCE}', \'crm_zcs_long_text_f\')}")'
find_replace(directory, 'WorkNote.py', findstr, replacestr)

findstr     = 'GetTable(f"{SOURCE}.crm_zcs_long_text_act")'
replacestr  = 'GetTable(f"{get_table_namespace(f'{SOURCE}', \'crm_zcs_long_text_act\')}")'
find_replace(directory, 'WorkNote.py', findstr, replacestr)

findstr     = 'cleansed.viewmaximosr'
replacestr  = '{get_table_namespace(\'cleansed\', \'maximo_viewmaximosr\')}'
find_replace(directory, 'maximo-view-creation.py', findstr, replacestr)

findstr     = 'cleansed.viewmaximowoactivity'
replacestr  = '{get_table_namespace(\'cleansed\', \'maximo_viewmaximowoactivity\')}'
find_replace(directory, 'maximo-view-creation.py', findstr, replacestr)

findstr     = '{SOURCE}.vw_aurion_employee_details'
replacestr  = '{get_table_namespace(f\'{SOURCE}\', \'aurion_employee_details\')}'
find_replace(directory, 'AurionEmployeeDetails.py', findstr, replacestr)

findstr     = 'hive_metastore.'
replacestr  = ''
find_replace(directory, 'AurionEmployeeDetails.py', findstr, replacestr)


Replacing "curated.view\<sourcesystem\>\<tablename\>" with "\<env_\>cleansed.\<schema\>.\<tablename\>_current" 
in notebooks under "/Databricks/MDP-Framework/Transform/Views/". These views are meant to be created in Cleansed layer.

In [0]:
directory   = "../Transform/Views/"

findstr     = 'view curated.view{i.DestinationSchema}{i.DestinationTableName}'
replacestr  = 'view {get_table_namespace(\'cleansed\', f\'{i.DestinationSchema}_{i.DestinationTableName}_current\')}'
find_replace(directory, '', findstr, replacestr)

print('Iteration 2')

findstr     = 'from cleansed.{i.DestinationSchema}_{i.DestinationTableName}'
replacestr  = 'from {get_table_namespace(\'cleansed\', f\'{i.DestinationSchema}_{i.DestinationTableName}\')}'
find_replace(directory, '', findstr, replacestr)

Replacing "\{target_schema\}.view\<tablename\>" with "\<env_\>\{target_schema\}.\<schema\>.\<tablename\>" 
in notebooks under "/Databricks/MDP-Framework/Transform/Views/". \{target_schema\} in MDP Framework is "curated", the target view names are looked up from mapping.csv.

In [0]:
directory   = "../Transform/Views/"

findstr     = '{target_schema}.view{table}'
replacestr  = '{get_table_namespace(f\'{target_schema}\', f\'{table}\')}'
find_replace(directory, '', findstr, replacestr)

print('Iteration 2')

findstr     = '{target_schema}.{table}'
replacestr  = '{get_table_namespace(f\'{target_schema}\', f\'{table}\')}'
find_replace(directory, '', findstr, replacestr)

Function to get the list of hardcoded cleansed tablenames in the view definitions, uses regex to pattern match 'cleansed.'

In [0]:
def get_strings_to_replace(full_path, pattern):
    strings_to_replace = []
    for filename in os.listdir(full_path):
        if filename.endswith(".py"):
            with open(os.path.join(full_path, filename), "r") as f:
                for line in f:
                    for word in line.split():
                        if pattern.search(word): 
                            if (len(word.split("."))) == 2:
                                strings_to_replace.append( re.sub( r'[)"](.*)', '', (re.sub('[^A-Za-z0-9.]+_', '', word)) ) )
                            else:
                                print (f"This was formatted poorly and is hence not included in the list:\n \'{filename} \'- {word}\n")
    return strings_to_replace

Finding and replacing the hardcoded cleansed table names in view definitions using the above function

In [0]:
directory = "../Transform/Views/"
patternstr = re.compile(r"\bcleansed.\b")

strings_to_replace =  get_strings_to_replace(full_path = directory, pattern = patternstr)

for findstr in sorted(strings_to_replace, key = len, reverse = True):
    # print(findstr)
    layer   = findstr.split(".")[0]
    schema  = findstr.split(".")[1].split("_")[0]
    table   = findstr.split(".")[1]
    replacestr  = f'{{get_table_namespace(\'{layer}\', \'{table}\')}}'
    # print(replacestr)
    find_replace(directory, '', findstr, replacestr)

In [0]:
directory = "../Transform/Views/"
patternstr = re.compile(r"{SOURCE}.")

strings_to_replace =  get_strings_to_replace(full_path = directory, pattern = patternstr)

for findstr in sorted(strings_to_replace, key = len, reverse = True):
    # print(findstr)
    layer   = findstr.split(".")[0]
    schema  = findstr.split(".")[1].split("_")[0]
    table   = findstr.split(".")[1]
    replacestr  = f'{{get_table_namespace(\'{layer}\', \'{table}\')}}'
    # print(replacestr)
    find_replace(directory, 'AurionEmployeeDetails.py', findstr, replacestr)

Replacing "curated.view\<name\>" with "\<env_\>curated.\<productdatabase\>.\<viewname\>" 
in notebooks under "/Databricks/MDP-Framework/Transform/Views/". These views are meant to be created in product specific database, resolved based on the csv lookup.

In [0]:
directory = "../Transform/Views/"
patternstr = re.compile(r"curated.dim")

strings_to_replace =  get_strings_to_replace(full_path = directory, pattern = patternstr)

for findstr in sorted(strings_to_replace, key = len, reverse = True):
    # print(findstr)
    layer   = findstr.split(".")[0]
    schema  = findstr.split(".")[1].split("_")[0]
    table   = findstr.split(".")[1]
    replacestr  = f'{{get_table_namespace(\'{layer}\', \'{table}\')}}'
    # print(replacestr)
    find_replace(directory, '', findstr, replacestr)

Replacing "curated_v3.view\<name\>" with "\<env_\>curated_v3.\<productdatabase\>.\<viewname\>" 
in notebooks under "/Databricks/MDP-Framework/Transform/Views/". These views are meant to be created in product specific database, resolved based on the csv lookup.

In [0]:
directory = "../Transform/Views/"
patternstr = re.compile(r"curated.view")

strings_to_replace =  get_strings_to_replace(full_path = directory, pattern = patternstr)

for findstr in sorted(strings_to_replace, key = len, reverse = True):
    # print(findstr)
    layer   = findstr.split(".")[0]
    schema  = findstr.split(".")[1].split("_")[0]
    table   = findstr.split(".")[1]
    replacestr  = f'{{get_table_namespace(\'{layer}\', \'{table}\')}}'
    # print(replacestr)
    find_replace(directory, '', findstr, replacestr)

In [0]:
directory = "../Transform/Views/"
patternstr = re.compile(r"curated_v3")

strings_to_replace =  get_strings_to_replace(full_path = directory, pattern = patternstr)

for findstr in sorted(strings_to_replace, key = len, reverse = True):
    # print(findstr)
    layer   = findstr.split(".")[0]
    schema  = findstr.split(".")[1].split("_")[0]
    table   = findstr.split(".")[1]
    replacestr  = f'{{get_table_namespace(\'{layer}\', \'{table}\')}}'
    # print(replacestr)
    find_replace(directory, '', findstr, replacestr)

In [0]:
directory = "../Transform/Views/"
patternstr = re.compile(r"curated_v2")

strings_to_replace =  get_strings_to_replace(full_path = directory, pattern = patternstr)

for findstr in sorted(strings_to_replace, key = len, reverse = True):
    # print(findstr)
    layer   = findstr.split(".")[0]
    schema  = findstr.split(".")[1].split("_")[0]
    table   = findstr.split(".")[1]
    replacestr  = f'{{get_table_namespace(\'{layer}\', \'{table}\')}}'
    # print(replacestr)
    find_replace(directory, '', findstr, replacestr)

In [0]:
directory = "../Transform/Views/"
patternstr = re.compile(r"{DEFAULT_TARGET}")

strings_to_replace =  get_strings_to_replace(full_path = directory, pattern = patternstr)

for findstr in sorted(strings_to_replace, key = len, reverse = True):
    # print(findstr)
    layer   = findstr.split(".")[0]
    schema  = findstr.split(".")[1].split("_")[0]
    table   = findstr.split(".")[1]
    replacestr  = f'{{get_table_namespace(f\'{layer}\', \'{table}\')}}'
    # print(replacestr)
    find_replace(directory, 'viewUnmeteredConsumption_UC2.py', findstr, replacestr)

adding f string where get_table_name function was called in place of hardcoded table names

In [0]:
directory = "../Transform/Views/"

findstr     = 'spark.sql("""'
replacestr  = 'spark.sql(f"""'
find_replace(directory, '', findstr, replacestr)